In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model 
import transformers
import torch
from datasets import load_dataset
from tqdm import tqdm

/home/cslab02/anaconda3/envs/sebas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-30 09:51:50.846272: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 09:51:50.888875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 09:51:50.888907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 09:

In [2]:
base_model = "/home/cslab02/Desktop/TesisSebasMena/text-generation-webui/loras/llama2-chat-7b-spanish-256-saved/" 


In [3]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit = True,
    device_map="auto",

)


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.48s/it]


In [5]:
ds = load_dataset(path="facebook/belebele",split="spa_Latn")

Extracting data files: 100%|██████████| 122/122 [00:00<00:00, 1433.05it/s]
Generating acm_Arab split: 900 examples [00:00, 30305.42 examples/s]
Generating arz_Arab split: 900 examples [00:00, 52428.07 examples/s]
Generating ceb_Latn split: 900 examples [00:00, 72287.89 examples/s]
Generating fin_Latn split: 900 examples [00:00, 68112.69 examples/s]
Generating hin_Deva split: 900 examples [00:00, 37671.89 examples/s]
Generating ita_Latn split: 900 examples [00:00, 71428.88 examples/s]
Generating khm_Khmr split: 900 examples [00:00, 33950.98 examples/s]
Generating lvs_Latn split: 900 examples [00:00, 52444.10 examples/s]
Generating npi_Deva split: 900 examples [00:00, 46740.71 examples/s]
Generating pol_Latn split: 900 examples [00:00, 64112.39 examples/s]
Generating slv_Latn split: 900 examples [00:00, 64959.71 examples/s]
Generating swe_Latn split: 900 examples [00:00, 63327.24 examples/s]
Generating tso_Latn split: 900 examples [00:00, 71225.37 examples/s]
Generating xho_Latn split: 9

In [6]:
# Select the first five rows of the dataset for example prompts
ds_examples=ds.select(range(0,5))
ds_prompts=ds.select(range(5,len(ds)))

prompt_template="""{flores_passage}
Question: {question}
Answer A: {mc_answer1}
Answer B: {mc_answer2}
Answer C: {mc_answer3}
Answer D: {mc_answer4}
Correct answer: {correct_answer}"""

# Prepare example prompts for 5-shot prompting
choices=["A","B","C","D"]
prompt_examples = "\n\n".join([ prompt_template.format(**d,correct_answer=choices[int(d["correct_answer_num"])-1]) for d in ds_examples])

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.2, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          )
          (k_proj): Linear8bitLt(
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.2, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features

In [15]:
tokenizer_path = "/home/cslab02/Desktop/TesisSebasMena/text-generation-webui/models/llama2-chat" 

In [16]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)


In [9]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [7]:
# parse model response and extract the model'schoice
def parse_choice(response):
    choices=["A","B","C","D"]
    
    if len(response)==1:
        return choices.index(response[0]) + 1 if response[0] in choices else None
    elif response[0] in choices and not response[1].isalpha():
        return choices.index(response[0]) + 1
    else:
        return None

# sampling parameters: llama-precise
gen_config = {
    "temperature": 0.7,
    "top_p": 0.1,
    "repetition_penalty": 1.18,
    "top_k": 40,
    "do_sample": True,
    "max_new_tokens": 5,
    "pad_token_id": pipeline.tokenizer.eos_token_id,
}

# Loop through prompts and evaluate model responses
q_correct = q_total = 0
for rowNo, row in enumerate(tqdm(ds_prompts)):        
    # Construct the prompt by combining the example prompts and the current row's question
    prompt=(prompt_examples + "\n\n" + prompt_template.format(**row, correct_answer="")).strip()

    # Generate a response from the model
    response=pipeline(prompt, **gen_config)[0]["generated_text"][len(prompt):]
    if "\n" in response:
        response=response.split("\n")[0]

    # Parse the model's choice and compare it to the correct answer
    choice=parse_choice(response.strip())
    if choice==int(row["correct_answer_num"]):
        q_correct+=1 
    q_total+=1

print(f"{q_total} questions, {q_correct} correct ({round(q_correct/q_total*100,1)}%)")  

AttributeError: 'function' object has no attribute 'tokenizer'

In [9]:
#model.load_adapter("/home/cslab02/Desktop/TesisSebasMena/text-generation-webui/loras/llama2-chat-7b-spanish-256-saved/")

AttributeError: 'LlamaForCausalLM' object has no attribute 'peft_config'

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)